In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## TODO!!(점수 올리기 위해서)
# 평가셋에서 점수가 잘 나오는 것으로 최적화
# 불안정
# 평가셋을 뽑을때도 제대로 뽑아야함<-테스트셋을 반영할 수 있게
# 테스트셋 데이터의 분포를 반영
# stratify<-classification분포에 맞게
# regression문제는? classification처럼 구간을 나눔
# 소금이 들어있는 비율로. (회귀문제를 분류모델처럼)
# 정답칼럼을 기준으로

# 직접 이미지를 가져올 때, size->101로, 모델 수정, 평가셋제대로 뽑기

In [ ]:
## 소금의 양에 따라 stratify하는 코드
# import zipfile

# #with 키워드 쓰면, close안써도 알아서 닫힘<-코드 줄 수 아낌
# with zipfile.ZipFile('/kaggle/input/tgs-salt-identification-challenge/train.zip','r') as z:
#     z.extractall('train') #train(.zip)폴더의 압축을 푼다.

# with zipfile.ZipFile('/kaggle/input/tgs-salt-identification-challenge/test.zip','r') as z :
#     z.extractall('test') #test(.zip)폴더의 압축을 푼다.

# from tqdm import tqdm_notebook

# #keras.preprocessing.image load_img로 이미지 확인
# #keras.preprocessing.image load_img와 PIL Image open은 같은 결과
# from keras.preprocessing.image import load_img, img_to_array
# from skimage.transform import resize
# import matplotlib.pyplot as plt


# train = pd.read_csv('/kaggle/input/tgs-salt-identification-challenge/train.csv')

# train.head()

# train['img'] = [np.array(load_img('train/images/'+x+'.png',grayscale=True))/255 for x in tqdm_notebook(train['id'])] # total_id <-이미지 불러오고 몇장인지 모를떄만 사용

# train['mask'] = [np.array(load_img('train/masks/'+x+'.png',grayscale=True))/255 for x in tqdm_notebook(train['id'])]

# train

# train[train['id']=='75efad62c1']['mask'].sum().sum()

# train['coverage'] =train['mask'].apply(np.sum) / 10201

# train['class']=(train['coverage']*10).astype(int)

# train['class']

# train['img'] #2차원
# reshape_img = np.asarray(train['img'].tolist()).reshape(-1,101,101,1)

# reshape_mask = np.asarray(train['mask'].tolist()).reshape(-1,101,101,1)

# reshape_mask.shape


# from sklearn.model_selection import train_test_split 
# x_train,x_valid,y_train,y_valid = train_test_split(reshape_img,reshape_mask,test_size=0.2,random_state=777,stratify=train['class']) #stratify의 분포에 맞게 split

In [ ]:
import zipfile

#with 키워드 쓰면, close안써도 알아서 닫힘<-코드 줄 수 아낌
with zipfile.ZipFile('/kaggle/input/tgs-salt-identification-challenge/train.zip','r') as z:
    z.extractall('train') #train(.zip)폴더의 압축을 푼다.

with zipfile.ZipFile('/kaggle/input/tgs-salt-identification-challenge/test.zip','r') as z :
    z.extractall('test') #test(.zip)폴더의 압축을 푼다.

In [ ]:
train_id = os.listdir('train/images')
test_id = os.listdir('test/images')

In [ ]:
# PIL로 이미지 확인
os.listdir('train')

os.listdir('train/masks')

from PIL import Image
x=Image.open('train/masks/182bfc6862.png')
import matplotlib.pyplot as plt
%matplotlib inline

x.size

os.listdir('train/images/')

x=Image.open('train/images/182bfc6862.png')
x

In [ ]:
#keras.preprocessing.image load_img로 이미지 확인
#keras.preprocessing.image load_img와 PIL Image open은 같은 결과
from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import matplotlib.pyplot as plt

plt.imshow(load_img('train/images/182bfc6862.png'))


plt.imshow(load_img('train/masks/182bfc6862.png'))

#흑백, 1차원
x=load_img('train/masks/182bfc6862.png')
x.size

#흑백, 1차원
x=load_img('train/images/182bfc6862.png')
x.size

In [ ]:
# /////////////////////// #

In [ ]:
img = Image.open('train/images/182bfc6862.png')
img = np.asarray(img)
img.shape

img # 3차원에 다 같은 값 들어있네->흑백이다.

img = img[:,:,0]

img2= Image.fromarray(img)
img2



mask = Image.open('train/masks/182bfc6862.png')
mask = np.asarray(img) #2차원이네


mask = Image.open('train/masks/89165071a4.png')
mask

In [ ]:
# # 이미지를 nparray로 가져왔다->이제부터는 nparray로 처리함 cf)그래서 sklearn가지고 resize함수 이용해도 상관 없는 거
# # 이미지 전처리, resize
# from tqdm import tqdm_notebook
# X_train = np.zeros((len(train_id),128,128,1),dtype=np.uint8)
# y_train = np.zeros((len(train_id),128,128,1),dtype=np.bool)
# for i , file_name in tqdm_notebook(enumerate(train_id),total=len(train_id)):
#     img = Image.open('train/images/'+file_name)
#     img = np.asarray(img)[:,:,0]
#     X_train[i] = resize(img,(128,128,1),mode = 'constant',preserve_range=True)
    
#     mask = Image.open('train/masks/'+file_name)
#     mask = np.asarray(img)[:,:]
#     y_train[i] = resize(img,(128,128,1),mode='constant',preserve_range=True)
    

In [ ]:
from tqdm import tqdm_notebook
X_train = np.zeros((len(train_id), 128, 128, 1), dtype=np.uint8)
y_train = np.zeros((len(train_id), 128, 128, 1), dtype=np.bool)
for i, file_name in tqdm_notebook(enumerate(train_id), total=len(train_id)):
    img = load_img('train/images/' + file_name)
    img = img_to_array(img)[:,:,0]
    X_train[i] = resize(img, (128, 128, 1), mode='constant', preserve_range=True)
    
    mask = load_img('train/masks/' + file_name)
    mask = img_to_array(mask)[:,:,0]
    y_train[i] = resize(mask, (128, 128, 1), mode='constant', preserve_range=True)

In [ ]:
y_train.shape

In [ ]:
#계속 image->np array로 바꾼 상태에서 처리중
#sklearn(머신러닝 툴킷, np array 처리 쉽게 가능)계속 쓸 거임
#train,val나누기 8:2 비율로

from sklearn.model_selection import train_test_split 
x_train,x_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state=777)

In [ ]:
#이미지 좌우반전
#np.fliplr() left right 반전(축 안 써줘도 알아서 잘 해줌)
x_train = np.append(x_train,[np.fliplr(x) for x in x_train],axis=0) # 행 기준으로 행 아래에 append해줌
y_train = np.append(y_train,[np.fliplr(y) for y in y_train],axis=0)

In [ ]:
from keras.models import Model
from keras.layers import *

In [ ]:
def convolution_block(x,filters,size,strides=(1,1),padding='same',activation=True):
    x = Conv2D(filters,size,strides=strides,padding=padding)(x)
    x = BatchNormalization()(x)
    if activation==True :
        x = Activation('relu')(x)
    return x

def residual_block(block_input,num_filters=16):
    x = Activation('relu')(block_input)
    x = BatchNormalization()(x)
    x = convolution_block(x,num_filters,(3,3))
    x = convolution_block(x,num_filters,(3,3),activation=False) # residual block 맨 마지막엔 activation x
    x = Add()([x,block_input]) # keras.layers에 있는 Add
    #x = x+block_input
    return x

In [ ]:
def build_model (input_layer,start_neuron,dropout_ratio=0.5):
    cnn1 = Conv2D(start_neuron*1,(3,3),padding='same',activation=None)(input_layer)
    cnn1 = residual_block(cnn1,start_neuron*1)
    cnn1 = residual_block(cnn1,start_neuron*1)
    cnn1 = Activation('relu')(cnn1)
    pool1 = MaxPool2D()(cnn1)
    pool1 = Dropout(dropout_ratio/2)(pool1)
    
    cnn2 = Conv2D(start_neuron*2,(3,3),padding='same',activation=None)(pool1)
    cnn2 = residual_block(cnn2,start_neuron*2)
    cnn2 = residual_block(cnn2,start_neuron*2)
    cnn2 = Activation('relu')(cnn2)
    pool2 = MaxPool2D()(cnn2)
    pool2 = Dropout(dropout_ratio)(pool2)
    
    cnn3 = Conv2D(start_neuron*4,(3,3),padding='same',activation=None)(pool2)
    cnn3 = residual_block(cnn3,start_neuron*4)
    cnn3 = residual_block(cnn3,start_neuron*4)
    cnn3 = Activation('relu')(cnn3)
    pool3 = MaxPool2D()(cnn3)
    pool3 = Dropout(dropout_ratio)(pool3)
    
    cnn4 = Conv2D(start_neuron*8,(3,3),padding='same',activation=None)(pool3)
    cnn4 = residual_block(cnn4,start_neuron*8)
    cnn4 = residual_block(cnn4,start_neuron*8)
    cnn4 = Activation('relu')(cnn4)
    pool4 = MaxPool2D()(cnn4)
    pool4 = Dropout(dropout_ratio)(pool4)
    
    cnnm = Conv2D(start_neuron*16,(3,3),padding='same',activation=None)(pool4)
    cnnm = residual_block(cnnm,start_neuron*16)
    cnnm = residual_block(cnnm,start_neuron*16)
    cnnm = Activation('relu')(cnnm)
    
    #block6 out shape:()
    cc4 = Conv2DTranspose(start_neuron*8,(3,3),padding='same',strides=(2,2))(cnnm) #Conv2DTrans=deconvolution(풀링하고,convolution하는 것). 여기서는 2배로 풀링한 후, 그것을 3x3필터로 convolution한다, 패딩을 했기 때문에 최종적으로는 크기가 2배가 된다.
    cc4 = concatenate([cc4,cnn4])
    cc4 = Dropout(dropout_ratio)(cc4)
    cnn4 = Conv2D(start_neuron*8,(3,3),activation=None,padding='same')(cc4)
    cnn4 = residual_block(cnn4,start_neuron*8)
    cnn4 = residual_block(cnn4,start_neuron*8)
    cnn4 = Activation('relu')(cnn4)
    
    #block6 out shape:()
    cc3 = Conv2DTranspose(start_neuron*4,(3,3),padding='same',strides=(2,2))(cnn4) #Conv2DTrans=deconvolution(풀링하고,convolution하는 것). 여기서는 2배로 풀링한 후, 그것을 3x3필터로 convolution한다, 패딩을 했기 때문에 최종적으로는 크기가 2배가 된다.
    cc3 = concatenate([cc3,cnn3])
    cc3 = Dropout(dropout_ratio)(cc3)
    cnn3 = Conv2D(start_neuron*4,(3,3),activation=None,padding='same')(cc3)
    cnn3 = residual_block(cnn3,start_neuron*4)
    cnn3 = residual_block(cnn3,start_neuron*4)
    cnn3 = Activation('relu')(cnn3)
    
    #block6 out shape:()
    cc2 = Conv2DTranspose(start_neuron*2,(3,3),padding='same',strides=(2,2))(cnn3) #Conv2DTrans=deconvolution(풀링하고,convolution하는 것). 여기서는 2배로 풀링한 후, 그것을 3x3필터로 convolution한다, 패딩을 했기 때문에 최종적으로는 크기가 2배가 된다.
    cc2 = concatenate([cc2,cnn2])
    cc2 = Dropout(dropout_ratio)(cc2)
    cnn2 = Conv2D(start_neuron*2,(3,3),activation=None,padding='same')(cc2)
    cnn2 = residual_block(cnn2,start_neuron*2)
    cnn2 = residual_block(cnn2,start_neuron*2)
    cnn2 = Activation('relu')(cnn2)
    
    #block6 out shape:()
    cc1 = Conv2DTranspose(start_neuron*1,(3,3),padding='same',strides=(2,2))(cnn2) #Conv2DTrans=deconvolution(풀링하고,convolution하는 것). 여기서는 2배로 풀링한 후, 그것을 3x3필터로 convolution한다, 패딩을 했기 때문에 최종적으로는 크기가 2배가 된다.
    cc1 = concatenate([cc1,cnn1])
    cc1 = Dropout(dropout_ratio)(cc1)
    cnn1 = Conv2D(start_neuron*1,(3,3),activation=None,padding='same')(cc1)
    cnn1 = residual_block(cnn1,start_neuron*1)
    cnn1 = residual_block(cnn1,start_neuron*1)
    cnn1 = Activation('relu')(cnn1)
    cnn1 = Dropout(dropout_ratio/2)(cnn1)
    
    output_layer = Conv2D(1,(1,1),activation='sigmoid')(cnn1)
    return output_layer
    



In [ ]:
input_layer = Input((128,128,1))
output_layer = build_model (input_layer,16,0.5)
model = Model(input_layer,output_layer)

In [ ]:
# # 모델링. U-net(핵심 : U자모양에서 대응되는 반대편을 채널의 뒤에 이어붙임)
# from keras.models import Model
# from keras.layers import *

# #block1 out shape:(64,64,8)
# inp = Input((128,128,1))
# cnn1 = Conv2D(8,(3,3),activation='relu',padding='same')(inp)
# cnn1 = Conv2D(8,(3,3),activation='relu',padding='same')(cnn1)
# pool1 = MaxPooling2D()(cnn1)

# #block2 out shape:(32,32,16)
# cnn2 = Conv2D(16,(3,3),activation='relu',padding='same')(pool1)
# cnn2 = Conv2D(16,(3,3),activation='relu',padding='same')(cnn2)
# pool2 = MaxPooling2D()(cnn2)

# #block3 out shape:(16,16,32)
# cnn3 = Conv2D(32,(3,3),activation='relu',padding='same')(pool2)
# cnn3 = Conv2D(32,(3,3),activation='relu',padding='same')(cnn3)
# pool3 = MaxPooling2D()(cnn3)

# #block4 out shape:(8,8,64)
# cnn4 = Conv2D(64,(3,3),activation='relu',padding='same')(pool3)
# cnn4 = Conv2D(64,(3,3),activation='relu',padding='same')(cnn4)
# pool4 = MaxPooling2D()(cnn4)

# #block5 out shape:(8,8,128)
# cnn5 = Conv2D(128,(3,3),activation='relu',padding='same')(pool4)
# cnn5 = Conv2D(128,(3,3),activation='relu',padding='same')(cnn5)

# #block6 out shape:()
# cc4 = Conv2DTranspose(64,(3,3),padding='same',strides=(2,2))(cnn5) #Conv2DTrans=deconvolution(풀링하고,convolution하는 것). 여기서는 2배로 풀링한 후, 그것을 3x3필터로 convolution한다, 패딩을 했기 때문에 최종적으로는 크기가 2배가 된다.
# cc4 = concatenate([cc4,cnn4])
# cnn4 = Conv2D(64,(3,3),activation='relu',padding='same')(cc4)
# cnn4 = Conv2D(64,(3,3),activation='relu',padding='same')(cnn4)

# #block7 out shape:()
# cc3 = Conv2DTranspose(32,(3,3),padding='same',strides=(2,2))(cnn4)
# cc3 = concatenate([cc3,cnn3])
# cnn3 = Conv2D(32,(3,3),activation='relu',padding='same')(cc3)
# cnn3 = Conv2D(32,(3,3),activation='relu',padding='same')(cnn3)

# #block8 out shape:()
# cc2 = Conv2DTranspose(16,(3,3),padding='same',strides=(2,2))(cnn3)
# cc2 = concatenate([cc2,cnn2])
# cnn2 = Conv2D(16,(3,3),activation='relu',padding='same')(cc2)
# cnn2 = Conv2D(16,(3,3),activation='relu',padding='same')(cnn2)

# #block9 out shape:()
# cc1 = Conv2DTranspose(8,(3,3),padding='same',strides=(2,2))(cnn2)
# cc1 = concatenate([cc1,cnn1])
# cnn1 = Conv2D(8,(3,3),activation='relu',padding='same')(cc1)
# cnn1 = Conv2D(8,(3,3),activation='relu',padding='same')(cnn1)

# out = Conv2D(1,(1,1),activation='sigmoid')(cnn1)
# model = Model(inp,out)

# model.summary()


In [ ]:
x_valid.shape

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
callbacks=[ModelCheckpoint("best.h5",save_best_only=True,verbose=1),EarlyStopping(patience=15),ReduceLROnPlateau(patience=8,factor=0.15,verbose=1)]

model.fit(x_train,y_train,validation_data=[x_valid,y_valid],batch_size=32,callbacks=callbacks,epochs=100)

In [ ]:
from tqdm import tqdm_notebook
X_test = np.zeros((len(test_id),128,128,1),dtype=np.uint8)
for i, filename in tqdm_notebook(enumerate(test_id),total=len(test_id)):
    img = Image.open('test/images/'+filename)
    img = np.asarray(img)[:,:,0]
    X_test[i] = resize(img,(128,128,1),mode='constant',preserve_range=True)



In [ ]:
model.load_weights("best.h5")

In [ ]:
preds_valid = model.predict(x_valid).reshape(-1,128,128)
y_valid = y_valid.reshape(-1,128,128)

def down(image): #이미지 원래 크기로 되돌림
    return resize(image,(101,101),mode='constant',preserve_range=True)

preds_valid = np.array([down(x) for x in preds_valid])
y_valid = np.array([down(y) for y in y_valid])

In [ ]:
# src: https://www.kaggle.com/aglotero/another-iou-metric
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]
#     print("intersection type:",type(intersection))
#     print("intersection shape:",intersection.shape)
#     print(intersection)
    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
#     print("area_true type:",type(area_true))
#     print("area_true shape:",area_true.shape)
#     print(area_true)
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
#     print("area_pred type:",type(area_pred))
#     print("area_pred shape:",area_pred.shape)
#     print(area_pred)
    area_true = np.expand_dims(area_true, -1) # axis=1(열)로 차원 확장 -> (2,)->(2,1)
#     print("expand area_true type:",type(area_true))
#     print("expand area_true shape:",area_true.shape)
#     print(area_true)
    area_pred = np.expand_dims(area_pred, 0) # axis =0(행)로 차원 호가장 ->(2,)->(1,2)
#     print("expandarea_pred type:",type(area_pred))
#     print("expand area_pred shape:",area_pred.shape)
#     print(area_pred)

    # Compute union
    union = area_true + area_pred - intersection
    #차원이 다 다른데 어떻게 연산을 하는 거지
#     print("union type:",type(union))
#     print("union shape:",union.shape)
#     print(union)

    # Exclude background from the analysis
#     print("-------------")
    intersection = intersection[1:,1:]
#     print("intersection type:",type(intersection))
#     print("intersection shape:",intersection.shape)
#     print(intersection)
    union = union[1:,1:]
#     print("union type:",type(union))
#     print("union shape:",union.shape)
#     print(union)
    union[union == 0] = 1e-9
#     print("union[union==0] type:",type(union))
#     print("union[union==0] shape:",union.shape)
#     print(union)

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
        
    return np.mean(metric)

In [ ]:
threshold = np.linspace(0,1,50) # 0~1을 50으로 나누어줌
ious = np.array([iou_metric_batch(y_valid, np.int32(preds_valid>x)) for x in tqdm_notebook(threshold)])

In [ ]:
threshold_best_index = np.argmax(ious)
threshold_best_index

In [ ]:
iou_best = ious[threshold_best_index]
iou_best

In [ ]:
threshold_best = threshold[threshold_best_index]
threshold_best

In [ ]:
plt.figure(figsize=(20,12))
plt.plot(threshold, ious)
plt.plot(threshold_best, iou_best, "xr", label="best threshold")
plt.title("Threshold vs IoU ({}, {})".format(threshold_best, iou_best))
plt.legend()

In [ ]:
result = model.predict(X_test, batch_size=128, verbose=1)

In [ ]:
from tqdm import trange
down_test = []
for i in trange(len(result)):
    down_test.append(resize(np.squeeze(result[i]), (101, 101), mode='constant', preserve_range=True))

In [ ]:
def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
%%time
result_dic = {j:rle_encode(np.round(down_test[i]>=threshold_best)) for i, j in enumerate(tqdm_notebook(os.listdir('test/images')))}

In [ ]:
sub = pd.DataFrame.from_dict(result_dic, orient='index')
sub.head()

In [ ]:
sub.index.names = ['id']
sub.head()

In [ ]:
sub.columns = ['rle_mask']
sub.head()

In [ ]:
sub = sub.reset_index()
sub.head()

In [ ]:
sub['id'] = sub['id'].apply(lambda x: x[:-4])

In [ ]:
sub.to_csv('sub.csv', index=False)